In [ ]:
import pandas as pd
import numpy as np
from PIL import Image, ExifTags
import torch
from tqdm import tqdm

from transformers import AutoModelForObjectDetection, AutoImageProcessor

In [ ]:
PATH = "logs/checkpoint-19208"
model = AutoModelForObjectDetection.from_pretrained(PATH)
image_processor = AutoImageProcessor.from_pretrained(PATH)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for flag in ExifTags.TAGS.keys():
	if ExifTags.TAGS[flag] == "Orientation":
		break


def load_image(image_path):
	image = Image.open(image_path)

	exif = image._getexif()
	if exif is not None:
		orientation = exif.get(flag, None)
		if orientation == 3:
			image = image.rotate(180, expand=True)
		elif orientation == 6:
			image = image.rotate(270, expand=True)
		elif orientation == 8:
			image = image.rotate(90, expand=True)
	return image

In [ ]:
from glob import glob

files = glob("dataset/images/val/*")
len(files)

In [ ]:
load_image(files[0])

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from torchvision.ops import nms

# Make predictions with thresholds and NMS
def predict_image(image_path, confidence_threshold=0.5, iou_threshold=0.5):
	# Load and process image
	image = load_image(image_path)
	inputs = image_processor(images=image, return_tensors="pt").to(device)
	
	# Make prediction
	with torch.no_grad():
		outputs = model(**inputs)
	
	# Process outputs with confidence threshold
	target_sizes = torch.tensor([image.size[::-1]]).to(device)
	results = image_processor.post_process_object_detection(
		outputs, 
		threshold=confidence_threshold,
		target_sizes=target_sizes
	)[0]
	
	# Apply NMS if needed
	boxes = results['boxes']
	scores = results['scores']
	labels = results['labels']
	
	# Apply NMS for each class
	keep_indices = []
	for label in labels.unique():
		class_mask = (labels == label)
		class_indices = torch.where(class_mask)[0]
		class_boxes = boxes[class_mask]
		class_scores = scores[class_mask]
		
		# Apply NMS
		class_keep = nms(class_boxes, class_scores, iou_threshold)
		keep_indices.extend(class_indices[class_keep].tolist())
	
	# Extract final detections
	final_boxes = boxes[keep_indices].cpu().numpy()
	final_scores = scores[keep_indices].cpu().numpy()
	final_labels = labels[keep_indices].cpu().numpy()
	
	return {
		'image': image,
		'boxes': final_boxes,
		'scores': final_scores,
		'labels': final_labels
	}

# Plot prediction results
def plot_predictions(results, figsize=(12, 10)):
	image = results['image']
	boxes = results['boxes']
	scores = results['scores']
	labels = results['labels']
	
	# Get class names
	id2label = model.config.id2label
	
	# Create figure and axes
	fig, ax = plt.subplots(figsize=figsize)
	ax.imshow(image)
	
	# Assign different colors to different classes
	colors = plt.cm.rainbow(np.linspace(0, 1, len(model.config.id2label)))
	
	# Plot each detection
	for box, score, label_id in zip(boxes, scores, labels):
		# Get color for this class
		color = colors[int(label_id) % len(colors)]
		
		# Create rectangle
		x1, y1, x2, y2 = box
		rect = patches.Rectangle(
			(x1, y1), x2-x1, y2-y1, 
			linewidth=2, 
			edgecolor=color, 
			facecolor='none'
		)
		ax.add_patch(rect)
		
		# Add label and score
		label_name = id2label[int(label_id)]
		ax.text(
			x1, y1-5, 
			f'{label_name}: {score:.2f}',
			color='white', 
			fontsize=10,
			bbox=dict(facecolor=color, alpha=0.7)
		)
	
	plt.axis('off')
	plt.tight_layout()
	return fig

In [ ]:
# Run prediction and visualization on a single image
def predict_and_visualize(image_path, confidence_threshold=0.5, iou_threshold=0.5):
	results = predict_image(
		image_path, 
		confidence_threshold=confidence_threshold, 
		iou_threshold=iou_threshold
	)
	fig = plot_predictions(results)
	plt.title(f'Predictions for {image_path.split("/")[-1]}')
	plt.show()
	return results

# Process multiple images
def process_multiple_images(image_paths, confidence_threshold=0.5, iou_threshold=0.5, max_images=5):
	# Process up to max_images
	paths_to_process = image_paths[:min(max_images, len(image_paths))]
	
	for image_path in tqdm(paths_to_process):
		predict_and_visualize(
			image_path, 
			confidence_threshold=confidence_threshold,
			iou_threshold=iou_threshold
		)

In [ ]:
predict_and_visualize(files[0], confidence_threshold=0.1, iou_threshold=0.7)

In [ ]:
predict_and_visualize(files[1], confidence_threshold=0.3, iou_threshold=0.7)

In [ ]:
from ipywidgets import interact, FloatSlider

# Interactive prediction with adjustable thresholds
def interactive_prediction(image_path):
	def predict_with_thresholds(confidence_threshold, iou_threshold):
		predict_and_visualize(
			image_path, 
			confidence_threshold=confidence_threshold,
			iou_threshold=iou_threshold
		)
	
	# Create interactive sliders
	interact(
		predict_with_thresholds,
		confidence_threshold=FloatSlider(min=0.1, max=0.9, step=0.1, value=0.5),
		iou_threshold=FloatSlider(min=0.1, max=0.9, step=0.1, value=0.5)
	)

# Example usage
# Try with a single image
sample_image_path = files[0]
# predict_and_visualize(sample_image_path, confidence_threshold=0.5, iou_threshold=0.5)

# Try interactive prediction
interactive_prediction(files[0])

# Process multiple images
# process_multiple_images(files, confidence_threshold=0.5, iou_threshold=0.5, max_images=3)

In [ ]:
interactive_prediction(files[10])

In [ ]:
from torchvision.ops import nms


# Make predictions with thresholds and NMS
def predict_images(image_paths, confidence_threshold=0.5, iou_threshold=0.5):
    # Load and process image
    images = [load_image(image_path) for image_path in image_paths]
    inputs = image_processor(images=images, return_tensors="pt").to(device)

    # Make prediction
    with torch.no_grad():
        outputs = model(**inputs)

    # Process outputs with confidence threshold
    target_sizes = torch.tensor([image.size[::-1] for image in images]).to(device)
    batch_results = image_processor.post_process_object_detection(
        outputs, threshold=confidence_threshold, target_sizes=target_sizes
    )

    for results, image in zip(batch_results, images):
        # Apply NMS if needed
        boxes = results["boxes"]
        scores = results["scores"]
        labels = results["labels"]

        # Apply NMS for each class
        keep_indices = []
        # for label in labels.unique():
        # 	class_mask = (labels == label)
        # 	class_indices = torch.where(class_mask)[0]
        # 	class_boxes = boxes[class_mask]
        # 	class_scores = scores[class_mask]

        # Apply NMS
        class_keep = nms(boxes, scores, iou_threshold)
        keep_indices = class_keep.tolist()

        # Extract final detections
        final_boxes = boxes[keep_indices].cpu().numpy()
        final_scores = scores[keep_indices].cpu().numpy()
        final_labels = labels[keep_indices].cpu().numpy()

        yield {
            "image": image,
            "boxes": final_boxes,
            "scores": final_scores,
            "labels": final_labels,
        }

In [ ]:
predictions = list(predict_images([files[0], files[1]], confidence_threshold=0.1, iou_threshold=0.3))

len(predictions)

In [ ]:
predictions[0]

In [ ]:
files[:2]

In [ ]:
batch_size = 16
predictions = [
    list(predict_images(
        files[i : i + batch_size], confidence_threshold=0.05, iou_threshold=0.3
    ))
    for i in tqdm(range(0, len(files), batch_size))
]

In [ ]:
predictions = sum(predictions, start=[])

len(predictions)

In [ ]:
predictions[0]

In [ ]:
type(predictions[0]["boxes"])

In [ ]:
label2id = {'anthracnose': 0, 'cssvd': 1, 'healthy': 2}
id2label = {v: k for k, v in label2id.items()}

In [ ]:
import pandas as pd
import os

# Convert predictions into a DataFrame
data = []
for file, prediction in zip(files, predictions):
	for box, score, label in zip(prediction['boxes'], prediction['scores'], prediction['labels']):
		data.append({
			'Image_ID': os.path.basename(file),
			# 'box': box.tolist(),
			'confidence': float(score),
			'class_id': int(label),
			'class': id2label[int(label)],
			"x_min": float(box[0]),
			"y_min": float(box[1]),
			"x_max": float(box[2]),
			"y_max": float(box[3]),
		})

df_predictions = pd.DataFrame(data)
df_predictions.head()

In [ ]:
df_predictions["Image_ID"].value_counts().describe()

In [ ]:
df_predictions.to_csv("dataset/validations/predictions.csv", index=False)